In [1]:
#Dependencies
import pandas as pd
import gensim #the library for Topic modelling
from gensim.models.ldamulticore import LdaMulticore
from gensim import corpora, models
import pyLDAvis.gensim_models #LDA visualization library

from nltk.corpus import stopwords
import string
from nltk.stem.wordnet import WordNetLemmatizer

import warnings
warnings.simplefilter('ignore')
from itertools import chain

In [2]:
# Import Module
import os

# Folder Path
path = "C:/Users/Admin/Desktop/ISB/Capstone/Work/Data/Sample"

col = ['topic','topic_text']
df1 = pd.DataFrame(columns = ['topic','topic_text'])
df1_list = []

# Change the directory
os.chdir(path)

# Read text File


def read_text_file(file_path):
    with open(file_path, 'r',encoding = 'unicode_escape') as f:
        return(f.read())


# iterate through all file
for file in os.listdir():
    # Check whether file is in text format or not
    if file.endswith(".txt"):
        df1_list = []
        file_path = f"{path}/{file}"
        print(file_path)
        df1_list.append(file)

        # call read text file function
        df1_list.append(read_text_file(file_path))
        df1 = df1.append(pd.DataFrame([df1_list],columns = col),ignore_index = True)
        
df1

C:/Users/Admin/Desktop/ISB/Capstone/Work/Data/Sample/mergefile.txt


,topic,topic_text
0,mergefile.txt,\nDATA SCICENCE FULL COURSE- 10 HOURS -EDUREKA...


In [3]:
#clean the data
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

def clean(text):
    stop_free = ' '.join([word for word in text.lower().split() if word not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = ' '.join([lemma.lemmatize(word) for word in punc_free.split()])
    return normalized.split()

In [4]:
df1['text_clean']= df1['topic_text'].apply(clean)

In [5]:
df1

,topic,topic_text,text_clean
0,mergefile.txt,\nDATA SCICENCE FULL COURSE- 10 HOURS -EDUREKA...,"[data, scicence, full, course, 10, hour, edure..."


In [6]:
#create dictionary
dictionary = corpora.Dictionary(df1['text_clean'])
#Total number of non-zeroes in the BOW matrix (sum of the number of unique words per document over the entire corpus).
print(dictionary.num_nnz)

10987


In [7]:
#create document term matrix
doc_term_matrix = [dictionary.doc2bow(doc) for doc in df1['text_clean'] ]
print(len(doc_term_matrix))

1


In [8]:
lda = gensim.models.ldamodel.LdaModel

In [9]:
num_topics=10
%time ldamodel = lda(doc_term_matrix,num_topics=num_topics,id2word=dictionary,passes=50,minimum_probability=0)

Wall time: 22.2 s


In [10]:
ldamodel.print_topics(num_topics=num_topics)

[(0,
  '0.000*"data" + 0.000*"so" + 0.000*"value" + 0.000*"let" + 0.000*"going" + 0.000*"1" + 0.000*"x" + 0.000*"learning" + 0.000*"number" + 0.000*"right"'),
 (1,
  '0.000*"data" + 0.000*"so" + 0.000*"right" + 0.000*"going" + 0.000*"value" + 0.000*"let" + 0.000*"see" + 0.000*"point" + 0.000*"one" + 0.000*"set"'),
 (2,
  '0.000*"data" + 0.000*"so" + 0.000*"going" + 0.000*"value" + 0.000*"let" + 0.000*"set" + 0.000*"one" + 0.000*"right" + 0.000*"learning" + 0.000*"1"'),
 (3,
  '0.000*"data" + 0.000*"so" + 0.000*"right" + 0.000*"one" + 0.000*"set" + 0.000*"value" + 0.000*"point" + 0.000*"1" + 0.000*"let" + 0.000*"going"'),
 (4,
  '0.000*"data" + 0.000*"so" + 0.000*"1" + 0.000*"let" + 0.000*"one" + 0.000*"variable" + 0.000*"going" + 0.000*"value" + 0.000*"number" + 0.000*"get"'),
 (5,
  '0.015*"data" + 0.014*"so" + 0.012*"one" + 0.008*"value" + 0.008*"right" + 0.008*"1" + 0.008*"going" + 0.007*"variable" + 0.007*"mean" + 0.007*"get"'),
 (6,
  '0.000*"data" + 0.000*"so" + 0.000*"1" + 0.000

In [11]:
lda_display = pyLDAvis.gensim_models.prepare(ldamodel, doc_term_matrix, dictionary, sort_topics=False, mds='mmds')
pyLDAvis.display(lda_display)

In [12]:
# Assigns the topics to the documents in corpus
lda_corpus = ldamodel[doc_term_matrix]

In [13]:
[doc for doc in lda_corpus]

[[(0, 4.95853e-07),
  (1, 4.95853e-07),
  (2, 4.95853e-07),
  (3, 4.95853e-07),
  (4, 4.95853e-07),
  (5, 0.15473391),
  (6, 4.95853e-07),
  (7, 4.95853e-07),
  (8, 4.95853e-07),
  (9, 0.8452621)]]

In [14]:
scores = list(chain(*[[score for topic_id,score in topic] \
                      for topic in [doc for doc in lda_corpus]]))

threshold = sum(scores)/len(scores)
print(threshold)

0.0999999958409262


In [15]:
cluster1 = [j for i,j in zip(lda_corpus,df1.index) if i[0][1] > threshold]
cluster2 = [j for i,j in zip(lda_corpus,df1.index) if i[1][1] > threshold]
cluster3 = [j for i,j in zip(lda_corpus,df1.index) if i[2][1] > threshold]
# cluster4 = [j for i,j in zip(lda_corpus,df.index) if i[3][1] > threshold]
# cluster5 = [j for i,j in zip(lda_corpus,df.index) if i[4][1] > threshold]

print(len(cluster1))
print(len(cluster2))
print(len(cluster3))
# print(len(cluster4))
# print(len(cluster5))


0
0
0


In [16]:
df1.iloc[cluster1]

,topic,topic_text,text_clean


In [17]:
df1.iloc[cluster2]

,topic,topic_text,text_clean


In [18]:
df1.iloc[cluster3]

,topic,topic_text,text_clean
